 To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

- **Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.**

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 

- **These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.**

- **If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**

- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

4. Submit a link to your Notebook on your Github repository. (10 marks)

Note: There are different website scraping libraries and packages in Python. For scraping the above table, you can simply use pandas to read the table into a pandas dataframe.

Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/

The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k

Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup


# 1. Scrapping and creating DataFrame

In [14]:
# Download dand parse data using BeautifulSoup :
# There were some problems with updated wikipedia page, so I turned to older version(Feb 2020) and copied new link
url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&diff=942851379&oldid=942655599'

# take text
source = requests.get(url).text

soup = BeautifulSoup(source,'lxml')
#print(soup.prettify())


In [3]:
# Let's try to have a look on a row of table
soup.tbody.tr.text

'\nPostcode\nBorough\nNeighbourhood\n'

**It shows us that we need to split rows with \n, and we will have to remove first and last space[1:-1]**

In [4]:
table = soup.find('tbody')
rows = table.find_all('tr')

#Extracting details
data = []
for row in rows:
    details = row.text.split('\n')[1:-1] 
    data.append(details)

In [6]:
# Convert list to dataFrame and take into consideration that first row is the name of columns
df = pd.DataFrame(data[1:], columns=data[0])
df = df.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'})# renaming columns

df.head(5)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2. Only processing the cells that have an assigned borough. Ignoring the cells with a borough that is Not assigned. Droping row where borough is "Not assigned"

In [7]:
df=df.drop(df[df.Borough == 'Not assigned'].index)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## 3.More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [10]:
# let's see any postal code with 2 or more neighborhoods
df[df['PostalCode']=='M6A']

,PostalCode,Borough,Neighborhood
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [15]:
# Combine them in one row seperated by ','
df=df.groupby(['PostalCode','Borough']).agg(','.join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [16]:
df['Neighborhood'].replace('Not assigned',df['Borough'],inplace=True)

## In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [13]:
df.shape

(103, 3)